# The MattingMen Proudly Present: Image Matting, Neural and Classical

### Download the Dataset

In [ ]:
import gdown
from zipfile import ZipFile
import pandas as pd
import os

def download_and_extract(zip_url, extract_to='./archive/', zip_path='temp.zip'):
    file_id = zip_url.split('/')[-2]
    drive_url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(drive_url, zip_path, quiet=False)
    
    with ZipFile(zip_path, 'r') as zip_ref: 
        zip_ref.extractall(extract_to)
    os.remove(zip_path)

google_drive_link = "https://drive.google.com/file/d/1ZWjgJ762I40bYn9DC9IDAsDbvwJ8Dqm5/view?usp=drive_link"
download_and_extract(google_drive_link)

#### Double Check That It's Working

In [1]:
def create_image_dataframe(root_dir):
    data = []
    
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if 'clip_' in subdir:
                image_type = 'clip'
            elif 'matting_' in subdir:
                image_type = 'matting'
            else:
                continue
                
            filename_without_extension = os.path.splitext(file)[0]
            full_path = os.path.join(subdir, file)
            
            data.append([filename_without_extension, full_path, image_type])
    
    df = pd.DataFrame(data, columns=['image_name', 'path', 'type'])
    return df

root_directory = './archive/'
image_df = create_image_dataframe(root_directory)

print(image_df.head())


            image_name                                               path  \
0  1803232244-00000010  ./archive/matting/1803232244/matting_00000000/...   
1  1803232244-00000428  ./archive/matting/1803232244/matting_00000000/...   
2  1803232244-00000372  ./archive/matting/1803232244/matting_00000000/...   
3  1803232244-00000399  ./archive/matting/1803232244/matting_00000000/...   
4  1803232244-00000158  ./archive/matting/1803232244/matting_00000000/...   

      type  
0  matting  
1  matting  
2  matting  
3  matting  
4  matting  
